In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="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"
   
    # other params...
)
response = llm.invoke("The first man to land on moon was..?")
print(response.content)


The first man to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.


In [2]:
import pandas as pd

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.livehire.com/careers/nikeinc/job/7G7YP/R8CTWY4YA5/data-scientistl")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






Data Scientist - Nike | LiveHire


















































Log inLiveHireLinkedInFacebookGoogleCancelPowered by‹ Back to all jobsData ScientistHybridLocationBeaverton, OR 97005 - United StatesWork TypeContract/TempPositions1 PositionPublished At:18 hours agoSQLAWSData AnalysispythonData ScienceTableauGitHubJiraPowerBIMachine LearningRefer someoneApplyOnsite: Mon-ThursCategory: TechnologyInnovative Technology; High Quality Products, Self-EmpowermentGlobally Responsible; Sustainable Products, Diversity of ThoughtCelebration of Sports; If You Have a Body, You are an AthleteTitle: Data Scientist Location: Beaverton, ORDuration: 2 year contractNIKE, Inc. does more than outfit the world's best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achievers, lea

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
print(res.content)

```
{
  "role": "Data Scientist",
  "experience": "1-4 years",
  "skills": [
    "Python", "SQL", "Data Analysis", "Data Science", "Tableau", "Machine Learning", "R", "Power BI", "GitHub", "Jira", "AWS"
  ],
  "description": "Lead to the accomplishment of key goals across the Global Tech (GT) Business Operations and Enablement (BOE) analytics functions. This role will be reporting to the Location: Beaverton, OR. The critical need is the development and work with key stakeholders across GT to understand requirements, develop sustainable data solutions, and provide insights and recommendations."
}
```


In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Scientist',
 'experience': '1-4 years',
 'skills': ['Python',
  'SQL',
  'Data Analysis',
  'Data Science',
  'Tableau',
  'Machine Learning',
  'R',
  'Power BI',
  'GitHub',
  'Jira',
  'AWS'],
 'description': 'Lead to the accomplishment of key goals across the Global Tech (GT) Business Operations and Enablement (BOE) analytics functions. This role will be reporting to the Location: Beaverton, OR. The critical need is the development and work with key stakeholders across GT to understand requirements, develop sustainable data solutions, and provide insights and recommendations.'}

In [9]:
df = pd.read_csv("my_portfolio.csv")

In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [16]:
# links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
# links
links = collection.query(query_texts=['python','react'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [17]:
job = json_res
job['skills']

['Python',
 'SQL',
 'Data Analysis',
 'Data Science',
 'Tableau',
 'Machine Learning',
 'R',
 'Power BI',
 'GitHub',
 'Jira',
 'AWS']

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Global Tech Business Operations

Dear Hiring Manager,

I came across the Data Scientist role at Global Tech Business Operations and Enablement (BOE) and was impressed by the organization's commitment to leveraging data-driven insights to drive key goals. As a Business Development Executive at AtliQ, I'd like to introduce our company as a trusted partner in delivering tailored data science solutions that can help you achieve your objectives.

At AtliQ, we specialize in developing sustainable data solutions that cater to the unique needs of our clients. Our team of experts has extensive experience in Python, SQL, Data Analysis, Data Science, Tableau, Machine Learning, R, Power BI, GitHub, Jira, and AWS – aligning perfectly with the skills required for the Data Scientist role. We've empowered numerous enterprises with our automated tools, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our portfoli